In [1]:
import random
import numpy as np
import pandas as pd
from copy import deepcopy
import data_generation as dg
import time

In [7]:
import import_ipynb
#we are importing the notebook files to avoid code redundancy
#python can do almost anything, except run fast
import s_metaheuristics as smt

In [10]:
import tabu_search as ts

In [119]:
graph, random_solution, total_edge_length = dg.read_scientific_instance('data\\nug_30_5.txt')
num_nodes = len(graph)

In [120]:
def tabu_local_search_combo(graph, solution, max_iterations, neighbor_gen_func, local_search_func, change_func ,tabu_tenure):
    current_solution = deepcopy(solution)
    current_value = dg.calculate_total_edge_length(graph, current_solution)
    best_solution = current_solution.copy()
    best_value = current_value
    tabu_list = []

    best_i = -1

    for ind in range(max_iterations):
        current_solution, _, _ = deepcopy(local_search_func(graph, current_solution, current_value, int(max_iterations*0.1), change_func))

        neighbors = neighbor_gen_func(current_solution)
        neighbors.sort(key=lambda solution: dg.calculate_total_edge_length(graph, solution))
        #print(neighbors[0], calculate_total_edge_length(graph, neighbors[0]))

        
        next_solution = None
        for neighbor in neighbors:
            if neighbor not in tabu_list and dg.calculate_total_edge_length(graph, neighbor) <= current_value:
                next_solution = neighbor
                break


        if next_solution is None:
            next_solution = neighbors[0]


        tabu_list.append(next_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)


        current_solution = deepcopy(next_solution)
        current_value = dg.calculate_total_edge_length(graph, current_solution)

        #update best solution
        if current_value < best_value:
            #print(current_value)
            best_i = ind
            #print('promena best_i', best_i)
            best_solution = deepcopy(current_solution)
            best_value = current_value

    return best_solution, best_value, best_i

In [121]:
#test random combo
#solution, value, iter = tabu_local_search_combo(graph, random_solution, 500, ts.tabu_inverse, smt.local_search, smt.make_change_scramble ,5)
#print(solution, value, iter)

In [122]:
#let's itterate through the best combinations, instead of manually calling each combo and remembering each result
#there was more possible combinations, but we used tabu tehniques that gave best results on their own (and we already have 29 here which is a testing nightmare)
neighbor_generators = [ts.tabu_swap_neighbors, ts.tabu_inverse, ts.tabu_inverse_complete, ts.tabu_scramble, ts.tabu_scramble_continuous]
change_functions = [smt.make_change_swap, smt.make_change_scramble, smt.make_change_inverse]
search_functions = [smt.local_search, smt.simulated_annealing]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

for neighbor_gen in neighbor_generators:
    for search in search_functions:
        for change in change_functions:

            start = time.time()    
            sol, val, iter = tabu_local_search_combo(graph,
                                          random_solution,
                                          500,
                                          neighbor_gen,
                                          search,
                                          change,
                                          5)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(neighbor_gen.__name__, search.__name__, change.__name__, ' : ', sol, val, iter, '\n')

            sols.append(sols)
            values.append(val)
            times.append(duration)

            methods.append(str(neighbor_gen.__name__ + ', ' + search.__name__ + ', ' + change.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(val), 'Time': duration})

            if val == best:
                if duration <= times[best_i]:    
                    best = val
                    best_i = number_of_combinations

            if val < best:
                best = val
                best_i = number_of_combinations

            average[0] += val
            average[1] += duration

            number_of_combinations += 1    



tabu_swap_neighbors local_search make_change_swap  :  [2, 24, 4, 23, 28, 0, 12, 26, 11, 7, 19, 16, 9, 21, 14, 15, 20, 8, 13, 10, 22, 29, 27, 17, 5, 25, 1, 6, 3, 18] 3416 19 

tabu_swap_neighbors local_search make_change_scramble  :  [0, 28, 12, 24, 11, 23, 26, 4, 2, 16, 19, 7, 21, 9, 14, 15, 8, 20, 13, 10, 22, 3, 5, 18, 17, 6, 1, 27, 29, 25] 3416 9 

tabu_swap_neighbors local_search make_change_inverse  :  [24, 11, 2, 0, 12, 28, 4, 26, 23, 16, 19, 7, 21, 9, 14, 15, 8, 20, 13, 22, 10, 29, 5, 25, 17, 27, 3, 18, 1, 6] 3416 17 

tabu_swap_neighbors simulated_annealing make_change_swap  :  [4, 23, 24, 12, 0, 28, 2, 26, 11, 19, 7, 16, 9, 21, 14, 15, 8, 20, 13, 22, 10, 1, 6, 18, 25, 5, 29, 3, 27, 17] 3416 45 

tabu_swap_neighbors simulated_annealing make_change_scramble  :  [11, 2, 24, 28, 4, 0, 12, 23, 26, 19, 16, 7, 9, 21, 14, 15, 20, 8, 13, 10, 22, 25, 18, 27, 1, 3, 17, 6, 29, 5] 3416 42 

tabu_swap_neighbors simulated_annealing make_change_inverse  :  [0, 2, 23, 28, 26, 4, 12, 24, 11, 7, 

In [123]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/tabu_local_combo.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/tabu_local_combo.csv'), index=False)

,Dim,Method,Value,Time
0,30,"tabu_swap_neighbors, local_search, make_change...",3416,7.87
1,30,"tabu_swap_neighbors, local_search, make_change...",3416,8.11
2,30,"tabu_swap_neighbors, local_search, make_change...",3416,7.74
3,30,"tabu_swap_neighbors, simulated_annealing, make...",3416,7.72
4,30,"tabu_swap_neighbors, simulated_annealing, make...",3416,8.32
5,30,"tabu_swap_neighbors, simulated_annealing, make...",3416,7.79
6,30,"tabu_inverse, local_search, make_change_swap",3416,7.75
7,30,"tabu_inverse, local_search, make_change_scramble",3416,8.17
8,30,"tabu_inverse, local_search, make_change_inverse",3416,7.67
9,30,"tabu_inverse, simulated_annealing, make_change...",3416,7.81


In [124]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: tabu_inverse, local_search, make_change_inverse 3416 7.67


,Dim,Method,Value,Time
0,30,"tabu_inverse, local_search, make_change_inverse",3416,7.67


In [125]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:

    df.loc[row_to_update.index, 'taloc_combo'] = values[best_i]
    df.loc[row_to_update.index, 'taloc_combo_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'taloc_combo': values[best_i], 'taloc_combo_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [126]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [3422.23, 30.81]


,Dim,Value,Time
0,30,3422.23,30.81


In [127]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    
    df.loc[row_to_update.index, 'taloc_combo'] = average[0]
    df.loc[row_to_update.index, 'taloc_combo_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'taloc_combo': average[0], 'taloc_combo_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)